In [ ]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud

In [ ]:
## import sklearn
import pandas as pd

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
fm.findSystemFonts()
plt.rcParams['font.family']= ["NanumGothicCoding"]
plt.rcParams["axes.unicode_minus"]=False
# GPU 환경 설정하기
# assert tf.test.is_gpu_available() == True, 'GPU 설정을 확인하세요.'
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_logical_devices('GPU'))

# 1. 데이터 준비
## 1-1. 데이터 가져오기

In [ ]:
# 데이터를 가져옵니다.
import pandas as pd
spam = pd.read_csv('spam.csv')

In [ ]:
spam['label']=spam['label'].replace(['ham', 'spam'],[0,1])

In [ ]:
# head 함수를 이용해 데이터를 확인해봅니다. (띄어쓰기가 되어 있지 않음을 알 수 있어요.)
spam.head()

# info() 함수를 이용해서 데이터의 정보를 확인합니다.
spam.info()

In [ ]:
spam.describe(include='O')

In [ ]:
# label 데이터 분포를 확인합니다.
spam['label'].unique()

# descibe 함수를 이용해서 기본 정보를 확인합니다.
spam.describe

## 1-3. 결측치 제거

In [ ]:
# 결측치를 제거한 후 확인합니다.
spam.isna().sum()

spam.dropna(axis=0, inplace=True)

spam.count()

spam['label'].value_counts().plot(kind='bar')

print('정상 메일과 스팸 메일의 개수')
print(spam.groupby('label').size().reset_index(name='count'))

print(f'정상 메일의 비율 = {round(spam["label"].value_counts()[0]/len(spam) * 100,3)}%')
print(f'스팸 메일의 비율 = {round(spam["label"].value_counts()[1]/len(spam) * 100,3)}%')

# 2. 텍스트 데이터 분석
## 2-1. 텍스트 길이 분포

In [ ]:
# 텍스트 길이 분포와 최대 길이를 확인합니다.
spam['text'].str.len()

spam['text'].str.len().max()

# konlpy를 활용하여 태깅 클래스를 불러옵니다.
import nltk
from konlpy.tag import Mecab
mecab = Mecab()

txt=' '.join(spam.text.explode())

nltk_spam = nltk.Text(' '.join(spam.text.explode()))
print(nltk_spam)

## 2-2. 형태소/명사 추출

In [ ]:
# 태깅 클래스를 활용하여 형태소/명사를 추출합니다.
morphs_spam = mecab.morphs(txt)
print(len(morphs_spam))
morphs_spam[:10]

nouns_spam = mecab.nouns(txt)
print(len(nouns_spam))
nouns_spam[:10]

pos_spam = mecab.pos(txt)
print(len(pos_spam))
print(pos_spam[:3])
print(mecab.pos(txt,flatten=False)[:3])
print(mecab.pos(txt,join=True)[:3])

## 2-3. NLTK Text로 변환



In [ ]:
from nltk import bigrams

nltk_morphs_spam = nltk.Text(morphs_spam)
nltk_nouns_spam = nltk.Text(nouns_spam)
print(nltk_nouns_spam)

## 2-4. Frequency plot

In [ ]:
# 형태소/명사 추출 각각 단어 분포를 확인합니다.
plt.figure(figsize=(8,4))
plt.title("형태소 개수 : " + str(len(nltk_morphs_spam.tokens)))
nltk_morphs_spam.plot(20)
plt.figure(figsize=(8,4))
plt.title("명사 개수 : " + str(len(nltk_nouns_spam.tokens)))
nltk_nouns_spam.plot(20)



## 2-5. Similar words

In [ ]:
# 어떤 단어(스팸, 주식...)의 유사 단어를 확인합니다.
nltk_morphs_spam.similar('주식')

In [ ]:
plt.figure(figsize=(10,5))
nltk_morphs_spam.dispersion_plot(['스팸','주식','광고',])
plt.show()

## 2-6. Vocabulary

In [ ]:
nltk_nouns_spam.vocab()

In [ ]:
# 단어 사전을 확인합니다.
def extended_extraction(text):
    pos=mecab.pos(text)
    extended_list=[]
    for i in range(len(pos)):
        if pos[i][1] in('NP','NNB','NNP','NNG','SF','SY'):
            extended_list.append(pos[i][0])
    return extended_list
token_list = extended_extraction(str(spam['text']))
token_list[:10]

from collections import Counter
vocab_collection = Counter(token_list)
print("고유한 토큰 갯수 : ",len(vocab_collection))
vocab = vocab_collection.most_common(len(vocab_collection))
print("빈도순으로 정렬 :",vocab)

## 2-7. Collocation

In [ ]:
# 연어(collocation)들을 확인합니다.
nltk_morphs_spam.collocations(window_size=5)
print("*"*50)
nltk_nouns_spam.collocations(window_size=5)

# 3. WordCloud
## 3-1. Morphs vs Nouns

In [ ]:
# 형태소와 명사 추출 각각에 대해 wordcloud로 확인합니다.
type(nltk_nouns_spam)

import matplotlib.font_manager as font_manager
from wordcloud import WordCloud
NGC_path = font_manager.findfont('NanumGothicCoding')
wc=WordCloud(font_path=NGC_path, max_font_size=40, background_color="white").generate(' '.join(nltk_nouns_spam))
plt.figure(figsize=(12,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

